In [ ]:
import subprocess
import neutralb1.utils as utils

WORKSPACE_DIR = utils.get_workspace_dir()

git_hash = subprocess.check_output(['git', 'rev-parse', 'HEAD'], cwd=WORKSPACE_DIR).decode('utf-8').strip()
print(git_hash)

# Input-Output Study
This will be the "standard" Input-Output, as it includes the same waveset that the MC was generated with, a spin-1 waveset with unconstrained D/S waves. We integrate over the entire $-t$ range to get enough statistics, and the validity of this is explored in [another notebook](./t_bin_comparison.ipynb). Here we will fit the accepted signal MC, and compare it to the "true" values that we generated, to see if the fit can resolve these values. The details of the generated signal MC, and thus this waveset, are:
* Generated with $b_1(1235)$ and $\rho(1450)$ Breit-Wigners fixed to their PDG values
  * an isotropic background is included, but is so small it's negligible
  * These fit results are from mass-independent fits (no Breit-Wigners) to the signal generated by these BW's.
* No `OmegaDalitz` amplitudes for changing the dalitz distribution of the $\omega$ and its corresponding $\lambda$ distribution
* No $D/S$ ratio that would be typically associated with the $b_1$
* Only in the PARA 0 orientation

Each bin of mass has 500 randomized fits and 100 bootstrap fits 

In [ ]:
# load common libraries
import pandas as pd
import pickle as pkl
import pathlib
import os, sys
import numpy as np
import matplotlib.pyplot as plt
from typing import Dict

# load neutralb1 libraries
import neutralb1.utils as utils
from neutralb1.analysis.result import ResultManager
import neutralb1.analysis.statistics as stats

utils.load_environment()

# load in useful directories as constants
CWD = pathlib.Path.cwd()
STUDY_DIR = f"{WORKSPACE_DIR}/studies/io-tests/spin-1/t_0.1-1.0/"

# set env variables for shell cells
os.environ["WORKSPACE_DIR"] = WORKSPACE_DIR
os.environ['STUDY_DIR'] = STUDY_DIR

In [ ]:
%%bash
# print out yaml file used to submit the fits
cat $STUDY_DIR/submission.YAML

In [ ]:
# load in preprocessed results
with open(f"{STUDY_DIR}/preprocessed_results_acceptance_corrected.pkl", "rb") as f:
    data = pkl.load(f)
    results = ResultManager(**data)

In [ ]:
results.summary()

## Analysis

### Standard Plots

In [ ]:
results.plot.intensity.jp()
plt.show()

In [ ]:
results.plot.intensity.waves()
plt.show()

In [ ]:
results.plot.intensity.waves(fractional=True)
plt.show()

In [ ]:
results.plot.intensity.waves(reflectivity="negative")
plt.savefig(f"{STUDY_DIR}/plots/waves_negative.pdf")

In [ ]:
results.plot.diagnostic.matrix()
plt.show()

In [ ]:
sig_moments = list(results.get_significant_moments(threshold=0.02))
results.plot.intensity.moments(moments=sig_moments)
plt.savefig(f"{STUDY_DIR}/plots/significant_moments.pdf")

### Naturalities

First we'll just do the total naturality contribution

In [ ]:
results.plot.intensity.plot(
    ["p", "m"], 
    fractional=True, 
    col_kwargs={"p": {"color":"red", "label":"Natural"}, "m": {"color":"blue", "label":"Unnatural"}})
plt.savefig(f"{STUDY_DIR}/plots/p_m_fractional.pdf")

### b1 and rho(1450) interference

In [ ]:
colors = plt.colormaps["Dark2"].colors # type: ignore # match colors to JP plot
results.plot.phase.mass_phase(
    "p1p0S", "p1mpP", 
    amp1_kwargs={"color": colors[2]},
    amp2_kwargs={"color": colors[3]},
    )
plt.savefig(f"{STUDY_DIR}/plots/interference.pdf")

Lets make a couple joyplots to see how the bootstrap distributions are shaped

In [ ]:
results.plot.bootstrap.joyplot(
    ["p1p0S", "p1mpP"],
)
plt.show()

In [ ]:
results.plot.bootstrap.joyplot(
    [results.phase_difference_dict[("p1p0S", "p1mpP")]]    
)
plt.show()

### Poor $1^+S_{+1}^{(+)}$ Wave Resolution
This wave seems to have trouble. Lets look at it with some other problematic waves in 3 sample mass bins across this problematic region

In [ ]:
last_index = results.get_fit_indices(1.22, 1.24)
print(last_index)

In [ ]:
indices = [0, 6, 11]
results.plot.bootstrap.pairplot(indices, ["p1ppS", "p1pmS", "p1pmD", "p1ppD"])


In the matrix plot, this wave does have some weird interference shape with another D wave

In [ ]:
results.plot.phase.mass_phase(
    "p1ppS", "p1pmD", fractional=True
)

### Moment to Wave Comparison
The H0_2000 wave seems to differ greatly between the truth and projected values. It's mathematical form is
$$
H^{0}(2,0,0,0) = \sum_\varepsilon 
-0.2|1^{-}P_{-1}^{(\varepsilon)}|^2
+ 0.2|1^{+}D_{-1}^{(\varepsilon)}|^2
-0.2|1^{-}P_{0}^{(\varepsilon)}|^2
+ 0.2|1^{+}D_{0}^{(\varepsilon)}|^2
-0.2|1^{-}P_{1}^{(\varepsilon)}|^2
+ 0.2|1^{+}D_{1}^{(\varepsilon)}|^2
\\
-0.565685	\Re\left[ 1^{+}S_{-1}^{(\varepsilon)}1^{+}D_{-1}^{(\varepsilon)\ast} \right]
-0.565685	\Re\left[ 1^{+}S_{0}^{(\varepsilon)}1^{+}D_{0}^{(\varepsilon)\ast} \right]
-0.565685	\Re\left[ 1^{+}S_{1}^{(\varepsilon)}1^{+}D_{1}^{(\varepsilon)\ast} \right]
$$

In [ ]:
results.plot.intensity.moments(moments=["H0_2000"])

We can reasonably drop the $1^-P_m^{(\varepsilon)}$ waves, as they are well resolved. The interference terms in the unnatural waves can be ignored, as those are dominated by error. This leaves us with the D-wave magnitudes, and the natural (D-S) interference terms

In [ ]:
fig, axs = plt.subplots(
    1, 3, figsize=(15, 5), sharex=True, sharey=True, layout="constrained"
)
base_waves = ["1pmD", "1p0D", "1ppD"]
for i, (ax, base_wave) in enumerate(zip(axs.flatten(), base_waves)):
    results.plot.intensity.plot(
        [f"p{base_wave}", f"m{base_wave}"],
        fractional=True,
        ax=ax,
        col_kwargs={
            f"p{base_wave}": {"color":"red", "label":"Natural"},
            f"m{base_wave}": {"color":"blue", "label":"Unnatural"},
        }
    )
    if i != 0:
        ax.get_legend().remove()
        ax.set_ylabel("")

In [ ]:
fig, axs = plt.subplots(
    2,
    3,
    sharex=True,        
    gridspec_kw={"wspace": 0.0, "hspace": 0.07},
    height_ratios=[3, 1],
    figsize=(30, 10),
    layout="constrained",
)

wave_pairs = [("p1pmS", "p1pmD"), ("p1p0S", "p1p0D"), ("p1ppS", "p1ppD")]
for i, (ax_top, ax_bottom, wave_pair) in enumerate(zip(axs[0], axs[1], wave_pairs)):
    results.plot.phase.mass_phase(
        wave_pair[0],
        wave_pair[1],        
        amp1_kwargs={"color": "tab:purple"},
        amp2_kwargs={"color": "tab:green"},
        amp_ax=ax_top,
        phase_ax=ax_bottom
    )

Instabilities in the fit align well with the moment, and seem to suggest that the bad fit results are not part of an ambiguous solution with the uniquely extractable values

### D/S ratio
We did not generate a D/S ratio in the signal MC, but we can check how much stronger the D wave is compared to the S wave, and compare to the truth information.

In [ ]:
sd_waves = ["m1pmD", "m1p0D", "m1ppD", "p1pmD", "p1p0D", "p1ppD"]
for wave in sd_waves:
    exclude_waves = [w for w in sd_waves if w != wave]
    results.plot.diagnostic.ds_ratio(exclude_waves=exclude_waves)
    plt.savefig(f"{STUDY_DIR}/plots/ds_ratio_{wave}.pdf")